使用上海商品交易所所有品种连续5天的成交量预测下一天所有品种的成交量
<br/>
训练集为2014年到2016年三年的上海个品种日总成交量数据
<br/>
测试集为2017年至今的个品种日成交量数据
<br/>
使用滑动窗口取样例，每次滑动一天，保证最大限度的采样

In [1]:
import MySQLdb, datetime
import numpy as np
from sklearn.neural_network import MLPRegressor

conn = MySQLdb.connect(host='219.224.169.45', user='lizimeng', passwd='codegeass', db='commodity', charset='utf8')
cursor = conn.cursor()

day1 = datetime.date(2014,1,1)
day2 = datetime.date.today() - datetime.timedelta(1)

commodities = []
days = []

cursor.execute('select distinct code from contract_daily where date between %s and %s', (day1,day2))
for row in cursor.fetchall():
    commodities.append(row)
    
cursor.execute('select distinct day from trade_day where day between %s and %s', (day1,day2))
for row in cursor.fetchall():
    days.append(row[0])
    
data = np.zeros((len(commodities), len(days)))

i = 0
for code in commodities:
    cursor.execute('select sum(volume) from contract_daily where code=%s and date between %s and %s group by date order by date asc', (code,day1,day2))
    j = 0
    for row in cursor.fetchall():
        data[i,j] = row[0]
        j += 1
    i += 1

cursor.close()
conn.close()

In [2]:
# 切分训练集和测试集合，2017年1月1日之前为训练集，之后为测试集
point = 0
while days[point]<=datetime.date(2016,12,31):
    point += 1

In [ ]:
nn = MLPRegressor(hidden_layer_sizes=(200,200), activation='logistic', max_iter=10**6)
# 训练神经网络
X = []
Y = []
for i in range(point-5):
    X.append(data[:,i:i+5].reshape(len(commodities)*5).tolist())
    Y.append(data[:,i+5])
nn.fit(X,Y)

In [ ]:
# 测试神经网络
X = []
Y = []
for i in range(point, len(days)-5):
    X.append(data[:,i:i+5].reshape(len(commodities)*5).tolist())
    Y.append(data[:,i+5])
print nn.score(X,Y)